In [ ]:
!pip install transformers datasets torchaudio jiwer

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 29.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 100.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 96.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 62.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 40.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 19.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 101.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
from datasets import load_dataset

dataset = load_dataset("victors3136/4000-it0.25-es0.25")


README.md:   0%|          | 0.00/525 [00:00<?, ?B/s]

train-00000-of-00005.parquet:   0%|          | 0.00/378M [00:00<?, ?B/s]

train-00001-of-00005.parquet:   0%|          | 0.00/379M [00:00<?, ?B/s]

train-00002-of-00005.parquet:   0%|          | 0.00/382M [00:00<?, ?B/s]

train-00003-of-00005.parquet:   0%|          | 0.00/382M [00:00<?, ?B/s]

...

val-00000-of-00001.parquet:   0%|          | 0.00/131M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/136M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/4800 [00:00<?, ? examples/s]

Generating val split:   0%|          | 0/400 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/400 [00:00<?, ? examples/s]

In [ ]:
from transformers.models.whisper import WhisperProcessor
import torchaudio
import torch

model_name = "openai/whisper-small"
processor = WhisperProcessor.from_pretrained(model_name)
def prepare_example(batch):
    audio = batch["audio"]

    waveform = torch.tensor(audio["array"], dtype=torch.float32)

    if audio["sampling_rate"] != 16_000:
        resampler = torchaudio.transforms.Resample(orig_freq=audio["sampling_rate"], new_freq=16_000)
        waveform = resampler(waveform)

    audio_array = waveform.numpy()

    batch["input_features"] = processor.feature_extractor(
        audio_array, sampling_rate=16_000
    ).input_features[0]

    batch["labels"] = processor.tokenizer(batch["sentence"]).input_ids

    return batch


preprocessor_config.json:   0%|          | 0.00/185k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/283k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/836k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.48M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/494k [00:00<?, ?B/s]

normalizer.json:   0%|          | 0.00/52.7k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/34.6k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.19k [00:00<?, ?B/s]

In [ ]:
dataset = dataset.map(prepare_example)

Map:   0%|          | 0/4800 [00:00<?, ? examples/s]

Map:   0%|          | 0/400 [00:00<?, ? examples/s]

Map:   0%|          | 0/400 [00:00<?, ? examples/s]

In [ ]:
from transformers.models.whisper import WhisperForConditionalGeneration

model = WhisperForConditionalGeneration.from_pretrained(model_name)

config.json:   0%|          | 0.00/1.97k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/967M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/3.87k [00:00<?, ?B/s]

In [ ]:
import torch
from dataclasses import dataclass
from typing import Any, Dict, List, Union

@dataclass
class DataCollatorSpeechSeq2Seq:
    processor: WhisperProcessor

    def __call__(self, features):
        input_features = [f["input_features"] for f in features]
        label_ids = [f["labels"] for f in features]

        batch = {
            "input_features": torch.tensor(input_features),
            "labels": processor.tokenizer.pad({"input_ids": label_ids}, return_tensors="pt").input_ids
        }
        return batch

data_collator = DataCollatorSpeechSeq2Seq(processor=processor)


In [ ]:
from huggingface_hub import whoami

print(whoami())

{'type': 'user', 'id': '67d03ad06c2fe1a01eaef018', 'name': 'victors3136', 'fullname': 'Victor Selegean', 'email': 'victor.v.selegean@gmail.com', 'emailVerified': True, 'canPay': False, 'periodEnd': None, 'isPro': False, 'avatarUrl': 'https://cdn-avatars.huggingface.co/v1/production/uploads/67d03ad06c2fe1a01eaef018/ObhjrUaydvZofwt2e2F-c.jpeg', 'orgs': [], 'auth': {'type': 'access_token', 'accessToken': {'displayName': 'colab_finetune', 'role': 'write', 'createdAt': '2025-05-10T14:49:34.983Z'}}}


In [ ]:
from transformers.trainer_seq2seq import Seq2SeqTrainer
from transformers.training_args_seq2seq import Seq2SeqTrainingArguments

training_args = Seq2SeqTrainingArguments(
    output_dir="./whisper-ro-multilingual-finetune-it25-es25",
    per_device_train_batch_size=8,
    learning_rate=1e-4,
    warmup_steps=500,
    num_train_epochs=10,
    logging_dir="./logs",
    save_total_limit=2,
    fp16=True,
    logging_steps=50,
)


trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=dataset["train"],
    eval_dataset=dataset["test"],
    tokenizer=processor.tokenizer,
    data_collator=data_collator,
)


<ipython-input-10-377346751fd7>:17: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(


In [ ]:
trainer.train()

Step,Training Loss
50,2.497200
100,1.006600
150,0.609200
200,0.472600
250,0.458300
300,0.517600
350,0.509300
400,4.476100
450,4.342100
500,4.085400


/usr/local/lib/python3.11/dist-packages/transformers/modeling_utils.py:3339: UserWarning: Moving the following attributes in the config to the generation config: {'max_length': 448, 'suppress_tokens': [1, 2, 7, 8, 9, 10, 14, 25, 26, 27, 28, 29, 31, 58, 59, 60, 61, 62, 63, 90, 91, 92, 93, 359, 503, 522, 542, 873, 893, 902, 918, 922, 931, 1350, 1853, 1982, 2460, 2627, 3246, 3253, 3268, 3536, 3846, 3961, 4183, 4667, 6585, 6647, 7273, 9061, 9383, 10428, 10929, 11938, 12033, 12331, 12562, 13793, 14157, 14635, 15265, 15618, 16553, 16604, 18362, 18956, 20075, 21675, 22520, 26130, 26161, 26435, 28279, 29464, 31650, 32302, 32470, 36865, 42863, 47425, 49870, 50254, 50258, 50360, 50361, 50362], 'begin_suppress_tokens': [220, 50257]}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
  warnings.warn(


TrainOutput(global_step=6000, training_loss=0.36746819559670985, metrics={'train_runtime': 8326.1109, 'train_samples_per_second': 5.765, 'train_steps_per_second': 0.721, 'total_flos': 1.385209921536e+19, 'train_loss': 0.36746819559670985, 'epoch': 10.0})

In [ ]:
trainer.push_to_hub("victors3136/whisper-ro-multilingual-finetune-it25-es25")

model.safetensors:   0%|          | 0.00/967M [00:00<?, ?B/s]

Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

training_args.bin:   0%|          | 0.00/5.43k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/victors3136/whisper-ro-multilingual-finetune-it25-es25/commit/1da10f6b6de74cc0e121e358eb84fee91614a987', commit_message='victors3136/whisper-ro-multilingual-finetune-it25-es25', commit_description='', oid='1da10f6b6de74cc0e121e358eb84fee91614a987', pr_url=None, repo_url=RepoUrl('https://huggingface.co/victors3136/whisper-ro-multilingual-finetune-it25-es25', endpoint='https://huggingface.co', repo_type='model', repo_id='victors3136/whisper-ro-multilingual-finetune-it25-es25'), pr_revision=None, pr_num=None)